# save some example tifs in a folder named test or rename my test folder in the for loop (should really make function of it)
Need to install osmnx and all packages in individual environment iwth strict channel priority
install everything from conda forge incompatibility runs strong in these packae

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio

     |████████████████████████████████| 19.3 MB 1.3 MB/s 


In [ ]:
!pip install utm

  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6109 sha256=67edf3744a5cad4de66fa7947e972f0db517ca66b7d2e7607419aecb23363eb1
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [ ]:
!pip install rioxarray

     |████████████████████████████████| 46 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.3 MB 8.8 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.8.0-py3-none-any.whl size=54247 sha256=696994a29f6f758822bf90629fcda4a20d1ee6da304088eba247084833563d76
  Stored in directory: /root/.cache/pip/wheels/0a/e5/fb/5ac6dcd7faf126d92239d03f0e7d6c46648094d7a8706d2a63
Successfully built rioxarray


In [ ]:
!pip install osmnx

     |████████████████████████████████| 95 kB 2.7 MB/s 
     |████████████████████████████████| 15.7 MB 124 kB/s 
     |████████████████████████████████| 62 kB 764 kB/s 
     |████████████████████████████████| 1.0 MB 58.4 MB/s 
     |████████████████████████████████| 994 kB 58.6 MB/s 
     |████████████████████████████████| 11.3 MB 48.3 MB/s 
     |████████████████████████████████| 11.2 MB 28.2 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matpl

In [ ]:
import pandas as pd
from multiprocessing import Pool
from tqdm.notebook import tqdm
import time
import numpy as np
import os
import rasterio
import glob as glob
import osmnx as ox
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
from osgeo import gdal, ogr, osr
import cv2
import subprocess
import shapely
from shapely.geometry import MultiLineString
from matplotlib.patches import PathPatch
import matplotlib.path
import pandas as pd
import sys
import utm
import zipfile
import rioxarray

In [ ]:
def get_road_buffer(gdf, ds, im_vis_file, output_raster,
                    buffer_meters=2, burn_value=1,
                    buffer_roundness=6,
                    plot_file='', figsize=(6, 6), fontsize=6,
                    dpi=800, show_plot=False,
                    verbose=False):
    '''
    Get buffer around roads defined by gdf_edges from osmnx rasterio dataset.
    Calls create_buffer_geopandas() and gdf_to_array().
    Returns geodataframe and output mask.
    '''

    gdf_buffer = create_buffer_geopandas(gdf, ds,
                                         buffer_distance=buffer_meters,
                                         buffer_roundness=buffer_roundness,
                                         projectToUTM=True)

    # create label image
    if len(gdf_buffer) == 0:
        mask_gray = np.zeros(cv2.imread(im_vis_file, 0).shape)
        cv2.imwrite(output_raster, mask_gray)
    else:
        gdf_to_array(gdf_buffer, im_vis_file, output_raster,
                     burn_value=burn_value)

    # load mask
    mask_gray = cv2.imread(output_raster, 0)

    return mask_gray, gdf_buffer


def create_buffer_geopandas(gdf,
                            ds,
                            buffer_distance=2,
                            buffer_roundness=1,
                            projectToUTM=True):
    '''
    Create a buffer around the lines of the geojson.
    Return a geodataframe.
    '''

    inGDF = gdf

    # set a few columns that we will need later
    try:
        inGDF['type'] = inGDF['highway'].values
    except:
        inGDF['type'] = inGDF['road_type'].values
    inGDF['class'] = 'highway'
    inGDF['highway'] = 'highway'

    if len(inGDF) == 0:
        return [], []

    # Transform gdf Roadlines into UTM so that Buffer makes sense
    if projectToUTM:
        tmpGDF = ox.project_gdf(inGDF)
    else:
        tmpGDF = inGDF
    gdf_utm_buffer = tmpGDF

    # perform Buffer to produce polygons from Line Segments
    # this creates a buffer around the lines with size buffer_distance
    # in meters
    # it connects tiles within radius buffer_roundness
    gdf_utm_buffer['geometry'] = tmpGDF.buffer(buffer_distance,
                                               buffer_roundness)
    #dissolve is basically a groupby for geopandas
    #combines/squashes all geometries into a single one
    gdf_utm_dissolve = gdf_utm_buffer.dissolve(by='class')
    gdf_utm_dissolve.crs = gdf_utm_buffer.crs

    if projectToUTM:
        gdf_buffer = gdf_utm_dissolve.to_crs(ds.crs)
    else:
        gdf_buffer = gdf_utm_dissolve

    return gdf_buffer

In [ ]:
def gdf_to_array(gdf, im_file, output_raster, burn_value=255):
    '''
    Turn geodataframe to array, save as image file with non-null pixels
    set to burn_value.
    Basically everywhere where there is a road = burn_value (default = white = 255), else 0.
    '''

    NoData_value = 0  # -9999

    gdata = gdal.Open(im_file)

    # set target info
    target_ds = gdal.GetDriverByName('GTiff').Create(output_raster,
                                                     gdata.RasterXSize,
                                                     gdata.RasterYSize, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform(gdata.GetGeoTransform())

    # set raster info
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(gdata.GetProjectionRef())
    target_ds.SetProjection(raster_srs.ExportToWkt())

    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(NoData_value)

    outdriver = ogr.GetDriverByName('MEMORY')
    outDataSource = outdriver.CreateDataSource('memData')
    tmp = outdriver.Open('memData', 1)
    outLayer = outDataSource.CreateLayer("states_extent", raster_srs,
                                         geom_type=ogr.wkbMultiPolygon)
    # burn
    burnField = "burn"
    idField = ogr.FieldDefn(burnField, ogr.OFTInteger)
    outLayer.CreateField(idField)
    featureDefn = outLayer.GetLayerDefn()
    for geomShape in gdf['geometry'].values:
        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(ogr.CreateGeometryFromWkt(geomShape.wkt))
        outFeature.SetField(burnField, burn_value)
        outLayer.CreateFeature(outFeature)
        outFeature = 0

    gdal.RasterizeLayer(target_ds, [1], outLayer, burn_values=[burn_value], options = ['ALL_TOUCHED=TRUE'])
    outLayer = 0
    outDatSource = 0
    tmp = 0

    return

In [ ]:
def reproject_tif(filename):
    #I tried projecting the gdf which works, but somehow gdal rasterio don't like it
    #that's wasteful but works
    #can overwrite reprojected tif again and again
    rds = rioxarray.open_rasterio(filename, decode_coords="all")
    rds_4326 = rds.rio.reproject("EPSG:4326")
    rds_4326.rio.to_raster("reprojected.tif")
    

In [ ]:
path = '/content/drive/MyDrive/ai4good/updated'
for file in sorted(glob.glob(path + '/*.tif')):
    reproject_tif(file)
    ds = rasterio.open('reprojected.tif')
    west, south, east, north = ds.bounds
    graph = ox.graph.graph_from_bbox(north, south, east, west)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(graph,
                                            nodes=True,
                                            edges=True,
                                            node_geometry=True,
                                            fill_edge_geometry=True)
    reprojected_tif = 'reprojected.tif'
    output = f'output_{file}.png'
    mask, gdf_buffer = get_road_buffer(gdf_edges, ds,
                                               im_vis_file=reprojected_tif,
                                               output_raster=output)
    ds.close()
    os.remove('reprojected.tif')
    plt.figure(figsize = (15,15))
    plt.imshow(mask)
    plt.show()

AttributeError: ignored